In [2]:
#if you still need to install beautifulsoup4

! pip install requests beautifulsoup4 lxml

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Basis-URL
base_url = "https://www.ots.at/suche"

# Anfrage-Parameter
params = {
    "query": "",
    "seite": 1,
    "emittentId": 195,  # Emittent-ID
    "startDate": 1136934001,  # Startdatum
    "endDate": 1200092399,    # Enddatum
    "channel": "index",
    "attachment": ""
}

# Funktion zum Abrufen von Artikeldetails
def scrape_article_content(article_url):
    try:
        response = requests.get(article_url)
        if response.status_code != 200:
            print(f"Fehler beim Abrufen der Artikel-URL: {article_url}")
            return None, None, None

        soup = BeautifulSoup(response.text, "html.parser")
        # Titel
        title = soup.find("h1").get_text(strip=True)
        # Inhalt
        content_div = soup.find("div", {"itemprop": "articleBody"})
        content = content_div.get_text(strip=True) if content_div else "Kein Inhalt verfügbar"
        # Veröffentlichungsdatum
        date_meta = soup.find("meta", itemprop="datePublished")
        date_time = soup.find("time", {"itemprop": "datePublished"})
        date = (
            date_meta["content"] if date_meta and date_meta.has_attr("content") 
            else date_time["datetime"] if date_time and date_time.has_attr("datetime") 
            else date_time.get_text(strip=True) if date_time 
            else "Kein Datum verfügbar"
        )
        return title, content, date
    except Exception as e:
        print(f"Fehler beim Abrufen der Details: {e}")
        return None, None, None

# Funktion zum Scrapen einer Seite
def scrape_page(page_number):
    params["seite"] = page_number
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print(f"Fehler beim Abrufen der Seite {page_number}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all("mat-card")
    data = []

    for article in articles:
        try:
            # Titel
            title_tag = article.find("h1", class_="display-3")
            title = title_tag.get_text(strip=True) if title_tag else "Kein Titel"

            # Teaser
            teaser_tag = article.find("p", class_="lead")
            teaser = teaser_tag.get_text(strip=True) if teaser_tag else "Kein Teaser"

            # Link
            link_tag = article.find("a", class_="link-detailed-view")
            link = f"https://www.ots.at{link_tag['href']}" if link_tag else "Kein Link"

            # Details scrapen
            article_title, article_content, article_date = scrape_article_content(link)

            # Daten hinzufügen
            data.append({
                "Titel": title,
                "Teaser": teaser,
                "Datum": article_date,
                "Link": link,
                "Inhalt": article_content
            })

        except Exception as e:
            print(f"Fehler beim Verarbeiten eines Artikels: {e}")
            continue

    return data

# Hauptfunktion für mehrere Seiten
def scrape_all_pages(start_page=1, end_page=5):
    all_data = []
    for page in range(start_page, end_page + 1):
        print(f"Scraping Seite {page}...")
        page_data = scrape_page(page)
        all_data.extend(page_data)
        time.sleep(2)  # Wartezeit zwischen Anfragen

    return all_data

# Ergebnisse speichern
def save_to_csv(data, filename="ots_scraper_data_spoe_partei.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Daten erfolgreich gespeichert in {filename}")




In [2]:
# Ausführung: Scrape data in steps
for i in range(5):  # Adjust the range as needed to cover all steps
    x = i * 50  # Calculate the starting page for this iteration
    if __name__ == "__main__":
        print(f"Scraping pages {x + 1} to {x + 50}...")
        scraped_data = scrape_all_pages(start_page=x + 1, end_page=x + 50)  
        if scraped_data:
            # Save to a unique file for each iteration
            filename = f"ots_scraper_data_spoe_partei_part_{i + 1}.csv"
            save_to_csv(scraped_data, filename)
        else:
            print(f"Keine Daten gescrapt für Seiten {x + 1} bis {x + 50}.")


Scraping pages 1 to 50...
Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Scraping Seite 7...
Scraping Seite 8...
Scraping Seite 9...
Scraping Seite 10...
Scraping Seite 11...
Scraping Seite 12...
Scraping Seite 13...
Scraping Seite 14...
Scraping Seite 15...
Scraping Seite 16...
Scraping Seite 17...
Scraping Seite 18...
Scraping Seite 19...
Scraping Seite 20...
Scraping Seite 21...
Scraping Seite 22...
Scraping Seite 23...
Scraping Seite 24...
Scraping Seite 25...
Scraping Seite 26...
Scraping Seite 27...
Scraping Seite 28...
Scraping Seite 29...
Scraping Seite 30...
Scraping Seite 31...
Scraping Seite 32...
Scraping Seite 33...
Scraping Seite 34...
Scraping Seite 35...
Scraping Seite 36...
Scraping Seite 37...
Scraping Seite 38...
Scraping Seite 39...
Scraping Seite 40...
Scraping Seite 41...
Scraping Seite 42...
Scraping Seite 43...
Scraping Seite 44...
Scraping Seite 45...
Scraping Seite 46...
Scraping Seite 47

In [4]:
df_spoe = pd.read_csv("ots_scraper_data_spoe_partei_part_1.csv")

display(df_spoe)



,Titel,Teaser,Datum,Link,Inhalt
0,Matznetter besorgt: US-Hypothekenkrise ist noc...,Veranstaltung des Renner-Instituts zum Einflus...,2008-01-11T16:09:52+01:00,https://www.ots.at/presseaussendung/OTS_200801...,Veranstaltung des Renner-Instituts zum Einflus...
1,Cap: SPÖ-Forderung nach Rückforderungsverzicht...,"Wien (SK) - Er begrüße, dass die Forderung der...",2008-01-11T15:31:39+01:00,https://www.ots.at/presseaussendung/OTS_200801...,"Wien (SK) - Er begrüße, dass die Forderung der..."
2,"Gusenbauer: Zur Pflegelösung kommt ""Aktion Sch...",Hacklerregelung bis 2013 verlängert - Pensions...,2008-01-11T15:21:04+01:00,https://www.ots.at/presseaussendung/OTS_200801...,Hacklerregelung bis 2013 verlängert - Pensions...
3,Kalina: Kanzler Gusenbauer zeigt Führungs- und...,"Hacklerregelung ""großer Erfolg für tausende ha...",2008-01-11T15:08:17+01:00,https://www.ots.at/presseaussendung/OTS_200801...,"Hacklerregelung ""großer Erfolg für tausende ha..."
4,Audio-OTS von Bundeskanzler Gusenbauer auf www...,Wien (SK) - Der Pressedienst der SPÖ stellt in...,2008-01-11T14:33:54+01:00,https://www.ots.at/presseaussendung/OTS_200801...,Wien (SK) - Der Pressedienst der SPÖ stellt in...
...,...,...,...,...,...
495,SPÖ-Termine vom 1. Oktober bis 7. Oktober 2007,Wien (SK) -,2007-09-28T11:32:13+02:00,https://www.ots.at/presseaussendung/OTS_200709...,"Wien (SK) -MONTAG, 1. Oktober 2007:Finanzstaat..."
496,Heute Freitag: SPÖ-Bundesgeschäftsführer Kalin...,"Wien (SK) - Wir erlauben uns, die VertreterInn...",2007-09-28T09:10:40+02:00,https://www.ots.at/presseaussendung/OTS_200709...,"Wien (SK) - Wir erlauben uns, die VertreterInn..."
497,Gusenbauer: Gemeinsames Vorangehen von Europa ...,Transatlantische Beziehungen neu definieren,2007-09-27T16:01:10+02:00,https://www.ots.at/presseaussendung/OTS_200709...,Transatlantische Beziehungen neu definierenWie...
498,Kalina: Altes Kabinett Schüssel Kabarettnummer,Reaktion Molterers auf Gorbachs Briefkopf-Affä...,2007-09-27T15:10:06+02:00,https://www.ots.at/presseaussendung/OTS_200709...,Reaktion Molterers auf Gorbachs Briefkopf-Affä...


In [5]:
import pandas as pd
import os

# Name der Ausgabe-Datei
output_filename = "ots_scraper_data_spoe_partei.csv"

def combine_csv_files(output_filename):
    # Aktuelles Verzeichnis des Notebooks
    current_directory = os.getcwd()

    # Liste, um alle DataFrames zu speichern
    all_data = []

    # Iteriere durch alle Dateien im aktuellen Verzeichnis
    for filename in os.listdir(current_directory):
        if filename.startswith("ots_scraper_data_spoe_partei_part_") and filename.endswith(".csv"):
            file_path = os.path.join(current_directory, filename)
            print(f"Verarbeite Datei: {file_path}")
            # Lade die CSV-Datei und füge sie zur Liste hinzu
            df = pd.read_csv(file_path)
            all_data.append(df)

    # Kombiniere alle DataFrames in ein einziges DataFrame
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        # Speichere das kombinierte DataFrame in einer neuen CSV-Datei
        combined_df.to_csv(output_filename, index=False, encoding="utf-8")
        print(f"Alle Dateien erfolgreich kombiniert in: {output_filename}")
    else:
        print("Keine Dateien gefunden, die kombiniert werden können.")

# Funktion ausführen
combine_csv_files(output_filename)


Verarbeite Datei: c:\Users\walchhth\OneDrive - Styria-IT Solutions GmbH & Co KG\Dokumente\KLZ - TOM\CSS - Master\1. Semester\Foundations of CSS\Group_Project\data\data_spoe\partei\ots_scraper_data_spoe_partei_part_1.csv
Verarbeite Datei: c:\Users\walchhth\OneDrive - Styria-IT Solutions GmbH & Co KG\Dokumente\KLZ - TOM\CSS - Master\1. Semester\Foundations of CSS\Group_Project\data\data_spoe\partei\ots_scraper_data_spoe_partei_part_2.csv
Verarbeite Datei: c:\Users\walchhth\OneDrive - Styria-IT Solutions GmbH & Co KG\Dokumente\KLZ - TOM\CSS - Master\1. Semester\Foundations of CSS\Group_Project\data\data_spoe\partei\ots_scraper_data_spoe_partei_part_3.csv
Verarbeite Datei: c:\Users\walchhth\OneDrive - Styria-IT Solutions GmbH & Co KG\Dokumente\KLZ - TOM\CSS - Master\1. Semester\Foundations of CSS\Group_Project\data\data_spoe\partei\ots_scraper_data_spoe_partei_part_4.csv
Verarbeite Datei: c:\Users\walchhth\OneDrive - Styria-IT Solutions GmbH & Co KG\Dokumente\KLZ - TOM\CSS - Master\1. Seme